In [7]:
import os, sys
import glob
import pandas as pd
from os import listdir
from os.path import isfile, join
from argparse import ArgumentParser

#Covarage matrix as csv file without qoutes, seperated by commas
file = "/home/pato/Skrivebord/HPV_subtyping/FASTQ/covMatrix.csv"
amplPos = "/home/pato/Skrivebord/HPV_subtyping/FASTQ/amplPos.bed"
fastqFiles = "/home/pato/Skrivebord/HPV_subtyping/FASTQ"

# Housegenes to check and remove
houseGeneEndRow = 5
typeTier=1

# Location of all valid reference names that can be given
path = "/home/pato/Skrivebord/HPV_subtyping/References"

# Location of where to save output files
refSavePath = "/home/pato/Skrivebord/HPV_subtyping/Scripts/Python/autoDetTest"
logFile = "/home/pato/Skrivebord/HPV_subtyping/Results/testlog.txt"


faF = [f for f in listdir(path) if isfile(join(path, f))]
faFiles = []
for i in faF:
    if "fasta.fai" not in i:
        faFiles.append(i)
df = pd.read_csv(file,sep=",")

housegenes = list(df.iloc[0:int(houseGeneEndRow),0])


In [4]:
excludeCols = ['Sort column', 'Gene', 'contig_srt', 'contig_end', 'Target']
sampleList = []
discardedList = []

for col in df.columns:
  
    if col not in excludeCols:

        # Checking housegenes, first get total cov of them
        hgTot = 0
        for gene in housegenes:
            
            hg = df[df['Gene']==gene]
            hgTot+= int(hg[col])

        # Now finding outliers in housegene coverage
        notEnoughCov = 0
        for gene in housegenes:
            
            hg = df[df['Gene']==gene]
            geneCov = int(hg[col])

            # set min cov on housegene to not count as failed
            if geneCov < hgTot*0.1:
                notEnoughCov+=1
            
        # discard sample if too many low housegenes
        if notEnoughCov < 4:
            sampleList.append(col)
        else:
            discardedList.append(col)
discardedList

[]

In [105]:
# Now remove housegenes from samples
ddf = df[~df['Gene'].isin(housegenes)]

In [106]:
ampl_df = pd.read_csv(amplPos,sep="\t", header = None)

In [107]:
# Getting amplicon id
amplList = []
for i in ddf['Gene']:
    amplList.append(i.split("_")[-1])
ddf['Target'] = amplList

<ipython-input-107-38ffdf97ff06>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ddf['Target'] = amplList


In [ ]:
ddf[['Type', 'Gene']] = ddf['Gene'].str.split('_', 1, expand=True)
# Removing Ampl from gene
# Getting amplicon id
GeneList = []
for i in ddf['Gene']:
    GeneList.append(i.split("AMPL")[~-1])
ddf['Gene'] = GeneList


In [120]:
ampl = pd.DataFrame()

In [121]:
ampl['Target'] = ampl_df[3]
ampl['contig_srt'] = ampl_df[1]
ampl['contig_end'] = ampl_df[2]

In [126]:
ddf = ddf.merge(ampl,on='Target')

In [127]:
# Look for deletions (possible integrations of the genome) by finding regions of very low coverage
hpvTypes = pd.unique(ddf['Type'])

for type in hpvTypes:
    filtered = ddf[ddf['Type'] == type]

In [138]:
delJson = {}

for sample in sampleList:
    for hpvtype in hpvTypes:
        filtered = ddf[ddf['Type'] == hpvtype]
        leng = len(filtered[sample])
        sum = filtered[sample].sum()
        
        # If sum of HPV type cover is less than n, do not call an integration
        if sum < 50:
            continue

        avg = sum / leng
        if avg > 50:
            cutoff = 50
        else:
            cutoff = avg*0.1

        #for sample in sampleList:#
        filtered = filtered.reset_index()
        lastIdx = 0
        for index, row in filtered.iterrows():
            
            if row[sample] < cutoff:
                # Check if last index contained low cov (potential deletion)
                if (lastIdx == index - 1) or (lastIdx == 0):

                    if 'startpos' not in locals():
                        startpos = row['contig_srt']
                    stoppos = row['contig_end']
                    lastIdx = index

                else:
                    # This is start of a new deletion
                    startpos = row['contig_srt']
                    lastrow = row
                    lastIdx = index


            else:
                # save found startpos and endpos
                if 'startpos' in locals():
                    if 'stoppos' not in locals():
                        stoppos = lastrow['contig_end']

                    # create key if it does not exist:
                    if sample in delJson.keys():
                        pass
                    else:    
                        delJson[sample] = {
                        'del' : []
                        }

                    delJson[sample]['del'].append([hpvtype,startpos,stoppos])

                    #save
                    del startpos, stoppos

In [139]:
delJson

{'IonXpress_003': {'del': [['HPV45', 194, 459]]},
 'IonXpress_005': {'del': [['HPV16', 2878, 6617]]},
 'IonXpress_006': {'del': [['HPV58', 744, 830]]},
 'IonXpress_008': {'del': [['HPV45', 194, 459],
   ['HPV45', 2878, 2958],
   ['HPV45', 3451, 3546]]},
 'IonXpress_012': {'del': [['HPV16', 149, 234],
   ['HPV16', 729, 808],
   ['HPV16', 3345, 3438]]},
 'IonXpress_015': {'del': [['HPV33', 383, 663], ['HPV33', 2853, 3558]]},
 'IonXpress_020': {'del': [['HPV39', 2894, 287]]},
 'IonXpress_024': {'del': [['HPV45', 373, 459], ['HPV45', 2878, 2958]]},
 'IonXpress_029': {'del': [['HPV16', 149, 234],
   ['HPV16', 575, 665],
   ['HPV16', 2870, 3438]]},
 'IonXpress_031': {'del': [['HPV35', 2853, 257], ['HPV35', 3323, 3405]]},
 'IonXpress_040': {'del': [['HPV16', 2878, 6617]]},
 'IonXpress_043': {'del': [['HPV33', 383, 663], ['HPV33', 3339, 3423]]},
 'IonXpress_046': {'del': [['HPV18', 130, 220], ['HPV18', 2857, 3676]]},
 'IonXpress_048': {'del': [['HPV31', 2878, 6617]]},
 'IonXpress_049': {'del':

In [141]:
integrationDF = pd.DataFrame(delJson)

In [142]:
filename = refSavePath+"/PossibleIntegrations.txt"

In [143]:
integrationDF.to_csv(filename)